# Position estimation from TDOA measurements

**Note: everything is scaled by $c = 3 \times 10^8$ (or we assume $c$ to be $1 \frac{m}{s}$)!**

In [ ]:
from collections import deque
import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sensors = np.array([
    [0.0, 0.0],
    [0.5, 0.0],
    [1.0, 0.0],
    [0.0, 0.5],
    [1.0, 0.5],
    [0.0, 1.0],
    [0.5, 1.0],
    [1.0, 1.0]])

n_sensors = sensors.shape[0]

sns.set_style('ticks')
plt.scatter(sensors[:, 0], sensors[:, 1])
plt.show()

In [ ]:
def toas(p):
    """Times of arrival at each sensor given position."""
    deltas = p[None, :] - sensors
    distances = np.linalg.norm(deltas, axis=1)
    return distances # recall c = 1


def d_toas(p):
    """Jacobian of toa by position."""
    deltas = p[None, :] - sensors
    distances = np.linalg.norm(deltas, axis=1)
    return deltas / distances[:, None]


def combinations(x):
    """Generate combinations for TDOA."""
    # Use first sensor as reference
    return itertools.product([x[0]], x[1:])


def differences(x):
    """Pairwise difference of combinations."""
    return np.array([t1 - t2 for t1, t2 in combinations(x)])


def covariance_matrix(variance):
    """Combination-difference covariance matrix.
    
    Args:
        variance: vector of variances of independent normally distributed random variables.
    
    Returns:
        covariance matrix of applying differences() to the n.d.r. variables.
    """
    
    def get_covariance(comb1, comb2):
        c = 0
        for i in (comb1 & comb2):
            c += variance[i]
        return c
    
    sensor_combs = [set(sensors) for sensors in combinations(range(n_sensors))]
    n_combs = len(sensor_combs)
    sensor_comb_combs = itertools.product(sensor_combs, sensor_combs)
    covariances = (get_covariance(c1, c2) for c1, c2 in sensor_comb_combs)
    return np.fromiter(covariances, dtype=np.float).reshape((n_combs, n_combs))

# Extended Kalman filter setup

No system model; we assume that we get noisy measurements of one constant position.

In [ ]:
def predict(p):
    """Predict TDOAs based on position."""
    return differences(toas(p))
    

def d_predict(p):
    """Jacobian of predict()."""
    return differences(d_toas(p))


def measure(truth, variance=np.ones(n_sensors)):
    """Noisy measurement given TOA variance for each sensor."""
    return differences(toas(truth) + np.random.randn(n_sensors) * variance)


def measure_cov(variance=np.ones(n_sensors)):
    """Measurement covariance matrix."""
    return covariance_matrix(variance)


def ekf_update(mean, cov, measure, measure_cov):
    z = measure
    R = measure_cov
    y = predict(mean)
    J = d_predict(mean)
    
    gain = cov.dot(J.T).dot(np.linalg.inv(J.dot(cov).dot(J.T) + R))
    
    return mean + gain.dot(z - y), (np.eye(len(mean)) - gain.dot(J)).dot(cov)


def estimate(init_mean, init_cov, truth, measure_covariance, iterations, seed):
    np.random.seed(seed)
    R = np.copy(measure_covariance)
    m = np.copy(init_mean)
    c = np.copy(init_cov)
    for _ in range(iterations):
        m, c = ekf_update(m, c, measure(truth), R)
        yield m, c

# Experiment

Let's set up some parameters.

In [ ]:
truth = np.array([0.5, 0.7])
init_mean = np.array([0.3, 0.2])
init_cov = np.eye(2) * 5
iterations = 400
np.random.seed(1234)
variance = 5 * np.random.rand(n_sensors)
measure_cov = covariance_matrix(variance)

def error(estimates):
    for mean, _ in estimates:
        yield np.linalg.norm(mean - truth)

# Preview

Preview some estimation processes.

In [ ]:
sns.set_style('darkgrid')
for seed in [0, 100, 200, 300, 400]:
    plt.plot(
        range(iterations),
        list(error(estimate(init_mean, init_cov, truth, measure_cov, iterations, seed))))
plt.show()

# Comparison
Compare with estimator that doesn't consider the TDOA-covariances.

In [ ]:
def error_with_covariance(seed):
    return error(estimate(init_mean, init_cov, truth, measure_cov, iterations, seed))


def error_variance_only(seed):
    measure_cov = np.diag([v1 + v2 for v1, v2 in combinations(variance)])
    return error(estimate(init_mean, init_cov, truth, measure_cov, iterations, seed))


plt.plot(range(iterations), list(error_with_covariance(123)), label='with covariance')
plt.plot(range(iterations), list(error_variance_only(123)), label='variance only')
plt.legend()
plt.show()

In [ ]:
def plot_convergence():

    def generate():
        seeds = range(100)
        for s in seeds:
            for t, e in enumerate(error_with_covariance(s)):
                yield t, s, 'with covariance', e
            for t, e in enumerate(error_variance_only(s)):
                yield t, s, 'variance only', e
            
    times = deque()
    units = deque()
    conditions = deque()
    values = deque()

    for t, u, c, v in generate():
        times.append(t)
        units.append(u)
        conditions.append(c)
        values.append(v)
        
    df = pd.DataFrame(dict(iteration=times, seed=units, method=conditions, error=values))

    sns.tsplot(data=df, time="iteration", unit="seed",
               condition="method", value="error")
    plt.show()
    
plot_convergence()

In [ ]:
def plot_final_error():
    
    def last(iterator):
        return deque(iterator, maxlen=1)[0]
    
    plt.scatter(
        range(100),
        list(last(error_with_covariance(s)) for s in range(100)),
        label='final error with covariance')
    plt.scatter(
        range(100),
        list(last(error_variance_only(s)) for s in range(100)),
        label='final error variance only')
    plt.legend()
    plt.show()

    
plot_final_error()